In [1]:
import pyautogui
import keyboard
import time
from mss import mss
from threading import Thread
import os
import json
from PIL import Image

class ActionLogger:
    def __init__(self, capture_rate=10, output_folder='output', resize_factor=3.16):
        self.capture_rate = capture_rate
        self.running = False
        self.keyboard_buffer = []
        self.output_folder = output_folder
        self.screen_counter = 0
        self.screen_timestamps = []
        self.resize_factor = resize_factor
        self.start_time = None
        self.end_time = None
        self.mouse_positions = []  # Add this line to store the mouse positions as an instance variable
        
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)


    def log_mouse_position(self):
        position = pyautogui.position()
        return {'x': position.x, 'y': position.y, 'time': time.time()}

    def log_screen(self):
        with mss() as sct:
            monitor = sct.monitors[0]
            screenshot = sct.grab(monitor)
            img = Image.frombytes("RGB", screenshot.size, screenshot.bgra, "raw", "BGRX")
            resized_img = img.resize((int(screenshot.width / self.resize_factor), int(screenshot.height / self.resize_factor)))
            filename = os.path.join(self.output_folder, f'screen_{self.screen_counter}.png')
            resized_img.save(filename)
            timestamp = time.time()
            self.screen_timestamps.append({'path': filename, 'time': timestamp})
            self.screen_counter += 1
            return filename

    def keyboard_listener(self):
        while self.running:
            event = keyboard.read_event()
            if event.event_type == keyboard.KEY_DOWN:
                self.keyboard_buffer.append({'name': event.name, 'time': event.time})

    def save_mouse_positions(self, mouse_positions):
        with open(os.path.join(self.output_folder, 'mouse_positions.json'), 'w') as f:
            json.dump(mouse_positions, f)

    def save_keyboard_buffer(self):
        with open(os.path.join(self.output_folder, 'keyboard_buffer.json'), 'w') as f:
            json.dump(self.keyboard_buffer, f)

    def save_screen_timestamps(self):
        metadata = {
            'screen_timestamps': self.screen_timestamps,
            'resize_factor': self.resize_factor,
            'original_screen_size': (pyautogui.size().width, pyautogui.size().height),
            'start_time': self.start_time,
            'end_time': self.end_time
        }
        with open(os.path.join(self.output_folder, 'screen_metadata.json'), 'w') as f:
            json.dump(metadata, f)

    def start(self):
        self.running = True
        self.start_time = time.time()
        self.keyboard_thread = Thread(target=self.keyboard_listener)
        self.keyboard_thread.start()

    def stop(self):
        self.running = False
        self.end_time = time.time()
        self.keyboard_thread.join()

    def run(self, capture_duration, save_every=60):
        self.start()
        
        start_time = time.time()
        last_save_time = start_time
        

        # Add this print statement to display the recording duration and output location
       


        while time.time() - start_time < capture_duration:
            current_time = time.time()
            time_elapsed = current_time - last_save_time
            recording_duration = current_time - start_time
            print(f"\rRecording for {recording_duration:.2f} seconds. Saving data to {self.output_folder}", end='', flush=True)

            self.mouse_positions.append(self.log_mouse_position())  # Update this line
            self.log_screen()
            
            time_elapsed = time.time() - last_save_time
            if time_elapsed >= save_every:
                self.save_mouse_positions(self.mouse_positions)  # Update this line
                self.save_keyboard_buffer()
                self.save_screen_timestamps()
                last_save_time = time.time()

            time.sleep(1 / self.capture_rate)

        self.stop()

        self.save_mouse_positions(self.mouse_positions)  # Update this line



In [2]:
import pyinput

In [3]:
import pyautogui
import keyboard
import time
from mss import mss
from threading import Thread
import os
import json
from PIL import Image

class ActionLogger:
    def __init__(self, capture_rate=10, output_folder='output', resize_factor=3.16):
        self.capture_rate = capture_rate
        self.running = False
        self.keyboard_buffer = []
        self.output_folder = output_folder
        self.screen_counter = 0
        self.screen_timestamps = []
        self.resize_factor = resize_factor
        self.start_time = None
        self.end_time = None
        
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

    def log_mouse_position(self):
        position = pyautogui.position()
        return {'x': position.x, 'y': position.y, 'time': time.time()}

    def log_screen(self):
        with mss() as sct:
            monitor = sct.monitors[0]
            screenshot = sct.grab(monitor)
            img = Image.frombytes("RGB", screenshot.size, screenshot.bgra, "raw", "BGRX")
            resized_img = img.resize((int(screenshot.width / self.resize_factor), int(screenshot.height / self.resize_factor)))
            filename = os.path.join(self.output_folder, f'screen_{self.screen_counter}.png')
            resized_img.save(filename)
            timestamp = time.time()
            self.screen_timestamps.append({'path': filename, 'time': timestamp})
            self.screen_counter += 1
            return filename

    def keyboard_listener(self):
        while self.running:
            event = keyboard.read_event()
            if event.event_type == keyboard.KEY_DOWN:
                self.keyboard_buffer.append({'name': event.name, 'time': event.time})

    def save_mouse_positions(self, mouse_positions):
        with open(os.path.join(self.output_folder, 'mouse_positions.json'), 'w') as f:
            json.dump(mouse_positions, f)

    def save_keyboard_buffer(self):
        with open(os.path.join(self.output_folder, 'keyboard_buffer.json'), 'w') as f:
            json.dump(self.keyboard_buffer, f)

    def save_screen_timestamps(self):
        metadata = {
            'screen_timestamps': self.screen_timestamps,
            'resize_factor': self.resize_factor,
            'original_screen_size': (pyautogui.size().width, pyautogui.size().height),
            'start_time': self.start_time,
            'end_time': self.end_time
        }
        with open(os.path.join(self.output_folder, 'screen_metadata.json'), 'w') as f:
            json.dump(metadata, f)

    def start(self):
        self.running = True
        self.start_time = time.time()
        self.keyboard_thread = Thread(target=self.keyboard_listener)
        self.keyboard_thread.start()

    def stop(self):
        self.running = False
        self.end_time = time.time()
        self.keyboard_thread.join()

    def run(self, capture_duration, save_every=60):
        self.start()
        
        mouse_positions = []
        start_time = time.time()
        last_save_time = start_time

        while time.time() - start_time < capture_duration:
            mouse_positions.append(self.log_mouse_position())
            self.log_screen()
            
            time_elapsed = time.time() - last_save_time
            if time_elapsed >= save_every:
                self.save_mouse_positions(mouse_positions)
                self.save_keyboard_buffer()
                self.save_screen_timestamps()
                last_save_time = time.time()

            time.sleep(1 / self.capture_rate)

        self.stop()

        self.save_mouse_positions(mouse_positions)


In [20]:
import pyautogui
import keyboard
import time
from mss import mss
from threading import Thread
import threading
import os
import json
from PIL import Image
from pynput import mouse


class ActionLogger:
    def __init__(self, capture_rate=10, output_folder='output', resize_factor=3.16):
        self.capture_rate = capture_rate
        self.running = False
        self.keyboard_buffer = []
        self.output_folder = output_folder
        self.screen_counter = 0
        self.screen_timestamps = []
        self.resize_factor = resize_factor
        self.start_time = None
        self.end_time = None
        self.mouse_positions = []
        self.mouse_clicked = False
        self.mouse_listener = None  # Add a member variable for the listener
        self.button = None  # Add a member variable for the button

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

    def log_mouse_position(self):
        position = pyautogui.position()
        timestamp = time.time()
        click = None
        if self.mouse_clicked:
            button_name = self.button.name  # Use the saved button attribute
            click = {'x': position.x, 'y': position.y, 'time': timestamp, 'button': button_name}
            self.mouse_clicked = False
        self.mouse_positions.append({'x': position.x, 'y': position.y, 'time': timestamp, 'click': click})

    def mouse_listener(self):
        def on_click(x, y, button, pressed):
            if pressed:
                self.button = button  # Save the button object as an attribute of the listener
                self.mouse_clicked = True

        listener = mouse.Listener(on_click=on_click)
        listener.start()
        self.mouse_listener = listener  # Save the listener object as an attribute

    def stop(self):
        self.running = False
        self.keyboard_thread.join(timeout=1)
        self.mouse_listener.stop()  # Call stop() on the listener object

    def run(self, capture_duration, save_every=60):
        self.start()
        
        start_time = time.time()
        last_save_time = start_time
        
        try:
            while time.time() - start_time < capture_duration:
                current_time = time.time()
                time_elapsed = current_time - last_save_time
                recording_duration = current_time - start_time
                print(f"\rRecording for {recording_duration:.2f} seconds. Saving data to {self.output_folder}", end='', flush=True)

                self.log_mouse_position()
                self.log_screen()

                time_elapsed = time.time() - last_save_time
                if time_elapsed >= save_every:
                    self.save_mouse_positions()
                    self.save_keyboard_buffer()
                    self.save_screen_timestamps()
                    last_save_time = time.time()

                time.sleep(1 / self.capture_rate)
        except KeyboardInterrupt:
            self.stop()

        self.save_mouse_positions()


    def log_screen(self):
        with mss() as sct:
            monitor = sct.monitors[0]
            screenshot = sct.grab(monitor)
            img = Image.frombytes("RGB", screenshot.size, screenshot.bgra, "raw", "BGRX")
            resized_img = img.resize((int(screenshot.width / self.resize_factor), int(screenshot.height / self.resize_factor)))
            filename = os.path.join(self.output_folder, f'screen_{self.screen_counter}.png')
            resized_img.save(filename)
            timestamp = time.time()
            self.screen_timestamps.append({'path': filename, 'time': timestamp})
            self.screen_counter += 1
            return filename

    def keyboard_listener(self):
        while self.running:
            event = keyboard.read_event()
            if event.event_type == keyboard.KEY_DOWN:
                self.keyboard_buffer.append({'name': event.name, 'time': event.time})

    def save_mouse_positions(self):
        with open(os.path.join(self.output_folder, 'mouse_positions.json'), 'w') as f:
            json.dump(self.mouse_positions, f)


    def save_keyboard_buffer(self):
        with open(os.path.join(self.output_folder, 'keyboard_buffer.json'), 'w') as f:
            json.dump(self.keyboard_buffer, f)

    def save_screen_timestamps(self):
        metadata = {
            'screen_timestamps': self.screen_timestamps,
            'resize_factor': self.resize_factor,
            'original_screen_size': (pyautogui.size().width, pyautogui.size().height),
            'start_time': self.start_time,
            'end_time': self.end_time
        }
        with open(os.path.join(self.output_folder, 'screen_metadata.json'), 'w') as f:
            json.dump(metadata, f)

    def start(self):
        self.running = True
        self.start_time = time.time()
        self.keyboard_thread = Thread(target=self.keyboard_listener)
        self.keyboard_thread.start()

        # Add the following lines to start the mouse listener thread
        self.mouse_thread = Thread(target=self.mouse_listener)
        self.mouse_thread.start()

    def stop(self):
        self.running = False
        self.keyboard_thread.join(timeout=1)
        self.mouse_thread.join(timeout=1)




    # def run(self, capture_duration, save_every=60):
    #     self.start()
        
    #     start_time = time.time()
    #     last_save_time = start_time
        
    #     try:
    #         while time.time() - start_time < capture_duration:
    #             current_time = time.time()
    #             time_elapsed = current_time - last_save_time
    #             recording_duration = current_time - start_time
    #             print(f"\rRecording for {recording_duration:.2f} seconds. Saving data to {self.output_folder}", end='', flush=True)

    #             self.log_mouse_position()
    #             self.log_screen()

    #             time_elapsed = time.time() - last_save_time
    #             if time_elapsed >= save_every:
    #                 self.save_mouse_positions()
    #                 self.save_keyboard_buffer()
    #                 self.save_screen_timestamps()
    #                 last_save_time = time.time()

    #             time.sleep(1 / self.capture_rate)
    #     except KeyboardInterrupt:
    #         self.stop()

    #     self.save_mouse_positions()  # Update this line



In [23]:
import pyautogui
import keyboard
import time
from mss import mss
from threading import Thread
import threading
import os
import json
from PIL import Image
from pynput import mouse


class ActionLogger:
    def __init__(self, capture_rate=10, output_folder='output', resize_factor=3.16):
        self.capture_rate = capture_rate
        self.running = False
        self.keyboard_buffer = []
        self.output_folder = output_folder
        self.screen_counter = 0
        self.screen_timestamps = []
        self.resize_factor = resize_factor
        self.start_time = None
        self.end_time = None
        self.mouse_positions = []
        self.mouse_clicked = False
        self.mouse_listener = None  # Add a member variable for the listener
        self.button = None  # Add a member variable for the button

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

    def log_mouse_position(self):
        position = pyautogui.position()
        timestamp = time.time()
        click = None
        if self.mouse_clicked:
            button_name = self.button.name  # Use the saved button attribute
            click = {'x': position.x, 'y': position.y, 'time': timestamp, 'button': button_name}
            self.mouse_clicked = False
        self.mouse_positions.append({'x': position.x, 'y': position.y, 'time': timestamp, 'click': click})

    def mouse_listener(self):
        def on_click(x, y, button, pressed):
            if pressed:
                click = {'name': f"{str(button.name)}+click", 'time': time.time()}
                self.keyboard_buffer.append(click)

        with mouse.Listener(on_click=on_click) as listener:
            listener.join()


    def stop(self):
        self.running = False
        self.keyboard_thread.join(timeout=1)
        self.mouse_listener.stop()  # Call stop() on the listener object

    def run(self, capture_duration, save_every=60):
        self.start()
        
        start_time = time.time()
        last_save_time = start_time
        
        try:
            while time.time() - start_time < capture_duration:
                current_time = time.time()
                time_elapsed = current_time - last_save_time
                recording_duration = current_time - start_time
                print(f"\rRecording for {recording_duration:.2f} seconds. Saving data to {self.output_folder}", end='', flush=True)

                self.log_mouse_position()
                self.log_screen()

                time_elapsed = time.time() - last_save_time
                if time_elapsed >= save_every:
                    self.save_mouse_positions()
                    self.save_keyboard_buffer()
                    self.save_screen_timestamps()
                    last_save_time = time.time()

                time.sleep(1 / self.capture_rate)
        except KeyboardInterrupt:
            self.stop()

        self.save_mouse_positions()


    def log_screen(self):
        with mss() as sct:
            monitor = sct.monitors[0]
            screenshot = sct.grab(monitor)
            img = Image.frombytes("RGB", screenshot.size, screenshot.bgra, "raw", "BGRX")
            resized_img = img.resize((int(screenshot.width / self.resize_factor), int(screenshot.height / self.resize_factor)))
            filename = os.path.join(self.output_folder, f'screen_{self.screen_counter}.png')
            resized_img.save(filename)
            timestamp = time.time()
            self.screen_timestamps.append({'path': filename, 'time': timestamp})
            self.screen_counter += 1
            return filename

    def keyboard_listener(self):
        while self.running:
            event = keyboard.read_event()
            if event.event_type == keyboard.KEY_DOWN:
                self.keyboard_buffer.append({'name': event.name, 'time': event.time})
            elif event.event_type == keyboard.KEY_UP:
                self.keyboard_buffer.append({'name': f"up {event.name}", 'time': event.time})

            # Fix for mouse button name
            elif event.event_type == keyboard.MOUSE_UP:
                button_name = 'left' if event.name == 'Button.left' else 'right' if event.name == 'Button.right' else 'middle'
                self.keyboard_buffer.append({'name': f"up {button_name}", 'time': event.time})
            elif event.event_type == keyboard.MOUSE_DOWN:
                button_name = 'left' if event.name == 'Button.left' else 'right' if event.name == 'Button.right' else 'middle'
                self.keyboard_buffer.append({'name': button_name, 'time': event.time})


    def save_mouse_positions(self):
        with open(os.path.join(self.output_folder, 'mouse_positions.json'), 'w') as f:
            json.dump(self.mouse_positions, f)


    def save_keyboard_buffer(self):
        with open(os.path.join(self.output_folder, 'keyboard_buffer.json'), 'w') as f:
            json.dump(self.keyboard_buffer, f)

    def save_screen_timestamps(self):
        metadata = {
            'screen_timestamps': self.screen_timestamps,
            'resize_factor': self.resize_factor,
            'original_screen_size': (pyautogui.size().width, pyautogui.size().height),
            'start_time': self.start_time,
            'end_time': self.end_time
        }
        with open(os.path.join(self.output_folder, 'screen_metadata.json'), 'w') as f:
            json.dump(metadata, f)

    def start(self):
        self.running = True
        self.start_time = time.time()
        self.keyboard_thread = Thread(target=self.keyboard_listener)
        self.keyboard_thread.start()

        # Add the following lines to start the mouse listener thread
        self.mouse_thread = Thread(target=self.mouse_listener)
        self.mouse_thread.start()

    def stop(self):
        self.running = False
        self.keyboard_thread.join(timeout=1)
        self.mouse_thread.join(timeout=1)
            

In [27]:
import pyautogui
import keyboard
import time
from mss import mss
from threading import Thread
import os
import json
from PIL import Image
from pynput import mouse


class ActionLogger:
    def __init__(self, capture_rate=10, output_folder='output', resize_factor=3.16):
        self.capture_rate = capture_rate
        self.running = False
        self.mouse_positions = []
        self.key_presses = []
        self.clicks = []
        self.output_folder = output_folder
        self.screen_counter = 0
        self.resize_factor = resize_factor

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

    def log_mouse_position(self, x, y):
        timestamp = time.time()
        self.mouse_positions.append({'name': f"mouse_position {x} {y}", 'time': timestamp})

    def on_click(self, x, y, button, pressed):
        if pressed:
            click = {'name': f"{str(button.name)}+click", 'time': time.time()}
            self.clicks.append(click)

    def stop(self):
        self.running = False
        self.keyboard_thread.join(timeout=1)
        self.mouse_listener.stop()

    def run(self, capture_duration, save_every=60):
        self.start()

        start_time = time.time()
        last_save_time = start_time

        try:
            while time.time() - start_time < capture_duration:
                current_time = time.time()
                time_elapsed = current_time - last_save_time
                recording_duration = current_time - start_time
                print(f"\rRecording for {recording_duration:.2f} seconds. Saving data to {self.output_folder}", end='', flush=True)

                self.log_screen()

                if time_elapsed >= save_every:
                    self.save_data()
                    last_save_time = time.time()

                time.sleep(1 / self.capture_rate)
        except KeyboardInterrupt:
            self.stop()

    def log_screen(self):
        with mss() as sct:
            monitor = sct.monitors[0]
            screenshot = sct.grab(monitor)
            img = Image.frombytes("RGB", screenshot.size, screenshot.bgra, "raw", "BGRX")
            resized_img = img.resize((int(screenshot.width / self.resize_factor), int(screenshot.height / self.resize_factor)))
            filename = os.path.join(self.output_folder, f'screen_{self.screen_counter}.png')
            resized_img.save(filename)
            self.screen_counter += 1
            return filename

    def keyboard_listener(self):
        while self.running:
            event = keyboard.read_event()
            if event.event_type == keyboard.KEY_DOWN:
                self.key_presses.append({'name': event.name, 'time': event.time})
            elif event.event_type == keyboard.KEY_UP:
                self.key_presses.append({'name': f"up {event.name}", 'time': event.time})

    def save_data(self):
        with open(os.path.join(self.output_folder, 'mouse_positions.json'), 'w') as outfile:
            json.dump(self.mouse_positions, outfile)

        with open(os.path.join(self.output_folder, 'key_presses.json'), 'w') as outfile:
            json.dump(self.key_presses, outfile)

        with open(os.path.join(self.output_folder, 'clicks.json'), 'w') as outfile:
            json.dump(self.clicks, outfile)

    def start(self):
        self.running = True
        self.keyboard_thread = Thread(target=self.keyboard_listener)
        self.keyboard_thread.start()

        self.mouse_listener = mouse.Listener(on_click=self.on_click, on_move=self.log_mouse_position)
        self.mouse_listener.start()

    def stop(self):
        self.running = False
        self.keyboard_thread.join(timeout=1)
        self.mouse_listener.stop()

In [28]:
logger = ActionLogger(output_folder='test6')
logger.run(capture_duration=300, save_every=10)  # Capture for 300 seconds (5 minutes)

Recording for 64.47 seconds. Saving data to test6